In [6]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw
import sys
from scipy.interpolate import interp1d

# Adding path to custom modules
sys.path.append("..")

from yolov5.detect import YOLODetector
from mathcalc import *

def calculate_angle(x_min, y_min, x_max, y_max, img_width, img_height, focal_length):
    # Calculate the center of the bounding box
    center_x = (x_min + x_max) / 2
    center_y = (y_min + y_max) / 2

    # Calculate the center of the image
    image_center_x = img_width / 2
    image_center_y = img_height / 2

    # Calculate the offsets
    delta_x = center_x - image_center_x
    delta_y = center_y - image_center_y

    # Calculate the angles
    theta_x = np.arctan(delta_x / focal_length)
    theta_y = np.arctan(delta_y / focal_length)

    # Convert radians to degrees
    theta_x_deg = np.degrees(theta_x)
    theta_y_deg = np.degrees(theta_y)

    return theta_x_deg, theta_y_deg 


def rotate_coordinates(xdata, ydata, zdata, angle_deg_x, angle_deg_y, angle_deg_z):
    # Convert angles from degrees to radians
    angle_rad_x = np.radians(angle_deg_x)
    angle_rad_y = np.radians(angle_deg_y)
    angle_rad_z = np.radians(angle_deg_z)

    # Calculate cos and sin for each angle
    cos_x, sin_x = np.cos(angle_rad_x), np.sin(angle_rad_x)
    cos_y, sin_y = np.cos(angle_rad_y), np.sin(angle_rad_y)
    cos_z, sin_z = np.cos(angle_rad_z), np.sin(angle_rad_z)

    # Rotate around X-axis
    ydata_rot = [cos_x * y - sin_x * z for y, z in zip(ydata, zdata)]
    zdata_rot = [sin_x * y + cos_x * z for y, z in zip(ydata, zdata)]
    ydata, zdata = ydata_rot, zdata_rot

    # Rotate around Y-axis
    xdata_rot = [cos_y * x + sin_y * z for x, z in zip(xdata, zdata)]
    zdata_rot = [-sin_y * x + cos_y * z for x, z in zip(xdata, zdata)]
    xdata, zdata = xdata_rot, zdata_rot

    # Rotate around Z-axis
    xdata_rot = [cos_z * x - sin_z * y for x, y in zip(xdata, ydata)]
    ydata_rot = [sin_z * x + cos_z * y for x, y in zip(xdata, ydata)]
    xdata, ydata = xdata_rot, ydata_rot

    return xdata, ydata, zdata

def calculate_trajectory(bspeed, langle, sprate, sangle, sspinr):
    updateGlobals(bspeed, langle, sprate, sangle, sspinr)

    # Initialize Data
    xpos = 0
    ypos = 0
    zpos = 0
    xdata = []
    ydata = []
    zdata = []

    InitVx, InitVy, InitVz = getInitialV()

    # Build Data Array
    for t in range(10000000):
        dt = 0.1  # 10ms increments
        dx, InitVx = calcX(dt, InitVx, InitVy, InitVz)
        dy, InitVy = calcY(dt, InitVy, InitVx, InitVz)
        dz, InitVz = calcZ(dt, InitVz, InitVx, InitVy)
        xpos = xpos + dx
        ypos = ypos + dy
        zpos = zpos + dz
        if ypos < 0:
            break
        xdata.append(xpos)
        ydata.append(ypos)
        zdata.append(zpos)
        calcSpinDecay(dt)

    # Convert data to yards
    xdata = meterstoyards(xdata)
    ydata = meterstoyards(ydata)
    zdata = meterstoyards(zdata)

    return xdata, ydata, zdata

def smooth_trajectory(xdata, ydata, zdata, num_points):
    t = np.linspace(0, 1, len(xdata))
    t_new = np.linspace(0, 1, num_points)

    x_interp = interp1d(t, xdata, kind='cubic')(t_new)
    y_interp = interp1d(t, ydata, kind='cubic')(t_new)
    z_interp = interp1d(t, zdata, kind='cubic')(t_new)

    return x_interp, y_interp, z_interp

def get_color_gradient(start_color, end_color, num_points):
    gradient = np.linspace(start_color, end_color, num_points)
    return gradient

def draw_trajectory_on_frame(frame, xdata, ydata, zdata, initial_point,
                            current_frame, start_frame, fps, total_flight_time,
                            bounding_box_width, sspinr, theta_x_deg, angle_start_end_point):
    if current_frame < start_frame:
        return frame

    image_size = (frame.shape[1], frame.shape[0])
    
    apex = round(max(ydata) * 3, 1)
    apex_x = round(max(xdata) * 3, 1)
    apex_z = round(xdata[-1],1)

    xdata, ydata, zdata = rotate_coordinates(xdata, ydata, zdata, angle_deg_x=theta_x_deg, angle_deg_y=angle_start_end_point, angle_deg_z=0)

    # Normalize data for image size
    max_x = max(xdata) if len(xdata) > 0 else 1
    max_y = max(ydata) if len(ydata) > 0 else 1
    max_z = max(zdata) if len(zdata) > 0 else 1

    if len(ydata) > 0:
        y = [int((y / max_y) * (2 * initial_point[1] + apex - image_size[1])) for y in ydata[:1]][0]
    initial_point_1 = initial_point[0]
    initial_point_2 = initial_point[1] + y

    xdata = [int((x / max_x) * (2 * initial_point_1 + apex_x - image_size[0])) for x in xdata]
    ydata = [int((y / max_y) * (2 * initial_point_2 + apex - image_size[1])) for y in ydata]
    zdata = [int((z / max_x) * (2 * initial_point_1 + apex_z - image_size[0])) for z in zdata]

    # Adjust to start from initial_point
    if sspinr > 0:
        zdata = [initial_point_1 + z for z in zdata]
        ydata = [initial_point_2 - y for y in ydata]  # Assuming the y-axis is inverted in image coordinates
    
    else:
        zdata = [initial_point_1- z for z in zdata]
        ydata = [initial_point_2 - y for y in ydata]  # Assuming the y-axis is inverted in image coordinates

    # Calculate the number of points to draw based on flying time
    current_time = (current_frame - start_frame) / fps
    total_frames = len(xdata)
    points_to_draw = int((current_time / total_flight_time) * total_frames)

    start_color = np.array([255, 255, 100])  # Bright yellow color
    end_color = np.array([255, 0, 255])  # Magenta color
    colors = get_color_gradient(start_color, end_color, len(xdata))

    max_thickness = bounding_box_width
    min_thickness = 4
    thickness_gradient = np.linspace(max_thickness, min_thickness, len(xdata))

    for i in range(min(len(xdata) - 1, points_to_draw)):
        color = tuple(map(int, colors[i]))
        thickness = int(thickness_gradient[i])
        cv2.line(frame, (zdata[i], ydata[i]), (zdata[i + 1], ydata[i + 1]), color, thickness)

    return frame

def xyxy2xywh(bbox):
    x1, y1, x2, y2 = bbox

    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2
    w = x2 - x1
    h = y2 - y1
    return [cx, cy, w, h]

if __name__ == '__main__':
    real_world_radius = 0.042672
    # Example input values
    video_path = './data/golf5.mp4'
    output_path = 'golf_with_trajectory.mp4'
    
    angle_start_end_point = 17.7
    bspeed, langle, sprate, sangle, sspinr = 110.6, 12.52, 2131.2, -1, 625 #golf5
    # bspeed, langle, sprate, sangle, sspinr = 135.2571, 12.3086, 2077.1382, -1, 913 #golf9
    # bspeed, langle, sprate, sangle, sspinr = 141.5, 22, 4128, -1, 981 #golf12
    
    # Initialize video capture and writer
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(cap.get(3)), int(cap.get(4))))

    # Initialize the detector
    detector = YOLODetector(weights='best.pt')
    boxes = detector.Prediction(video_path)
    
    if not boxes:
        print("No bounding boxes detected.")
        sys.exit(1)

    bbox = boxes[0]
    start_x = (bbox[0] + bbox[2]) // 2
    start_y = (bbox[1] + bbox[3]) // 2
    initial_point = (start_x, start_y)
    bounding_box_width = bbox[2] - bbox[0]
    
    focal = 1000.0
    bbox_xywh = xyxy2xywh(bbox)
    focal_length = focal / 256 * bbox_xywh[2]
    
    # Calculate angles
    theta_x_deg, theta_y_deg  = calculate_angle(bbox[0], bbox[1], bbox[2], bbox[3], frame_width, frame_height, focal_length)
    theta_x_deg = 90-theta_x_deg-1
    # theta_x_deg = 16
    
    print(theta_x_deg )

    # Calculate trajectory
    xdata, ydata, zdata = calculate_trajectory(bspeed, langle, sprate, sangle, sspinr)
    # Smooth trajectory over the total number of frames
    xdata, ydata, zdata = smooth_trajectory(xdata, ydata, zdata, total_frames)

    total_flight_time = 4.2  # Total flight time in seconds (example value)
    start_frame = int(total_flight_time * fps)  # Start drawing after 1.5 seconds
    end_frame = total_frames - int(0.01 * fps)  # Stop drawing 1.2 seconds before the end

    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        # cv2.imwrite('frame9.jpg', frame)
        if not ret:
            break

        if frame_index >= start_frame:
            # Draw trajectory on frame
            frame = draw_trajectory_on_frame(frame, xdata, ydata, zdata, initial_point,
                                            frame_index, start_frame, fps, total_flight_time, 
                                            bounding_box_width, sspinr, theta_x_deg, angle_start_end_point)

        out.write(frame)
        frame_index += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()


Fusing layers... 
Model summary: 212 layers, 22323858 parameters, 0 gradients, 52.6 GFLOPs


16.97725307046997
